In [202]:
import os
import json
import pandas as pd
import traceback

In [203]:
from langchain import PromptTemplate, LLMChain
# from langchain_cohere import ChatCohere
from langchain_cohere.chat_models import ChatCohere
from langchain_core.messages import HumanMessage

In [204]:
from dotenv import load_dotenv
load_dotenv()

True

In [205]:
KEY = os.getenv('COHERE_API_TOKEN')

In [206]:
llm = ChatCohere(cohere_api_key=KEY,model="command-r-plus",temperature=0.8)

In [207]:
# llm2 =ChatCohere(cohere_api_key=KEY,model="rerank-english-v3.0",temperature=0.8)

In [208]:
from langchain.chains import SequentialChain
import PyPDF2

In [209]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [210]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [211]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below and use it as a guide and make sure that the formate is in json  that are able to loads in json. \
Do not wrtie json and \\n in response. Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [212]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [213]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [214]:
TEMPLATE2="""
For each of the following multiple-choice questions, provide a comprehensive review of the options and select the best answer. Explain your reasoning for each selection and provide additional context or examples to support your choices. 
Subject: {subject}
MCQ: {quiz}
Review and Selection:
[Explain why one of the options is the best answer, providing supporting evidence and a clear rationale. Offer additional insights or examples related to the subject matter to enhance the explanation.]
Remember to tailor your explanation to the specific subject and question asked, offering a detailed and insightful review for each option before making your selection. Justify your choices thoroughly and feel free to provide alternative perspectives or discuss any limitations to the question or options presented.
"""

In [215]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [216]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [217]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [218]:
file_path = r"C:\Users\shail\Desktop\Projects\mcq_generator\data.txt"

In [219]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [220]:
NUMBER=10
SUBJECT="biology"
TONE="simple"

In [221]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [222]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [223]:
quiz=response.get("quiz")

In [224]:
quiz = json.loads(quiz)

In [225]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [226]:
quiz_table_data

[{'MCQ': 'What is biology?',
  'Choices': 'a: The study of the natural environment. | b: The scientific study of life. | c: A branch of science that deals with the study of chemicals. | d: The study of the universe and its phenomena.',
  'Correct': 'b'},
 {'MCQ': 'What are the unifying themes that tie biology together as a field?',
  'Choices': 'a: Cell structure, evolution, energy processing, and environmental regulation. | b: Chemistry, physics, and biology. | c: Cell structure and function. | d: Evolution and genetics only.',
  'Correct': 'a'},
 {'MCQ': 'What do all organisms have in common?',
  'Choices': 'a: They all have a similar life span. | b: They are made up of cells. | c: They can all move and grow. | d: They can all reproduce.',
  'Correct': 'b'},
 {'MCQ': 'What is the role of energy processing in living organisms?',
  'Choices': 'a: It helps in the regulation of internal processes. | b: It allows for movement, growth, and reproduction. | c: It is essential for the functio

In [227]:
quiz=pd.DataFrame(quiz_table_data)


In [228]:
quiz.to_csv("mcq_questions.csv",index=False)
